# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe2ec9589d0>
├── 'a' --> tensor([[-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288]])
└── 'x' --> <FastTreeValue 0x7fe2ec9cce20>
    └── 'c' --> tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                        [-0.8661,  0.3360, -0.4622,  0.5913],
                        [ 0.2358, -0.9527, -0.7433,  0.7739]])

In [4]:
t.a

tensor([[-2.0548, -0.2632, -0.3245],
        [ 1.0957,  0.3798, -0.1288]])

In [5]:
%timeit t.a

74.4 ns ± 0.317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe2ec9589d0>
├── 'a' --> tensor([[1.1294, 0.4543, 0.3333],
│                   [1.8006, 1.8185, 0.6213]])
└── 'x' --> <FastTreeValue 0x7fe2ec9cce20>
    └── 'c' --> tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                        [-0.8661,  0.3360, -0.4622,  0.5913],
                        [ 0.2358, -0.9527, -0.7433,  0.7739]])

In [7]:
%timeit t.a = new_value

78.6 ns ± 0.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288]]),
    x: Batch(
           c: tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                      [-0.8661,  0.3360, -0.4622,  0.5913],
                      [ 0.2358, -0.9527, -0.7433,  0.7739]]),
       ),
)

In [10]:
b.a

tensor([[-2.0548, -0.2632, -0.3245],
        [ 1.0957,  0.3798, -0.1288]])

In [11]:
%timeit b.a

67.5 ns ± 0.324 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1765,  0.8144,  1.3414],
               [-0.7930,  1.1215, -1.5349]]),
    x: Batch(
           c: tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                      [-0.8661,  0.3360, -0.4622,  0.5913],
                      [ 0.2358, -0.9527, -0.7433,  0.7739]]),
       ),
)

In [13]:
%timeit b.a = new_value

597 ns ± 9.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

942 ns ± 3.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 4.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

161 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe2ec953130>
├── 'a' --> tensor([[[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]],
│           
│                   [[-2.0548, -0.2632, -0.3245],
│                    [ 1.0957,  0.3798, -0.1288]]])
└── 'x' --> <FastTreeValue 0x7fe2ec953220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.1 µs ± 391 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe24160d430>
├── 'a' --> tensor([[-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288],
│                   [-2.0548, -0.2632, -0.3245],
│                   [ 1.0957,  0.3798, -0.1288]])
└── 'x' --> <FastTreeValue 0x7fe241676fa0>
    └── 'c' --> tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                        [-0.8661,  0.3360, -0.4622,  0.5913],
                 

In [23]:
%timeit t_cat(trees)

34.7 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

62.9 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]],
       
               [[-2.0548, -0.2632, -0.3245],
                [ 1.0957,  0.3798, -0.1288]]]),
    x: Batch(
           c: tensor([[[ 0.1846, -0.1902, -2.2872,  0.3354],
                       [-0.8661,  0.3360, -0.4622,  0.5913],
                       [ 0.2358, -0.9527, -0.7433,  0.7739]],
         

In [26]:
%timeit Batch.stack(batches)

88.2 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288],
               [-2.0548, -0.2632, -0.3245],
               [ 1.0957,  0.3798, -0.1288]]),
    x: Batch(
           c: tensor([[ 0.1846, -0.1902, -2.2872,  0.3354],
                      [-0.8661,  0.3360, -0.4622,  0.5913],
                      [ 0.2358, -0.9527, -0.7433,  0.7739],
                      [ 0.1846, -0.1902, -2.2872,  0.3354],
                      [-0.8661,  

In [28]:
%timeit Batch.cat(batches)

166 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

382 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
